In [ ]:
# Datani topib o‘qish (CSV/ZIP)

# Text column’ni tozalash (REAC_pt_symptom_v2_clean)

# y_* larni tekshirish (0/1, NaN yo‘q, y_sum)

# y_labels ni y_* dan qayta yasash (agar xohlasangiz; konsistensiya uchun)

# 0-label audit (soni, top terms)

# Saqlash (processed CSV)

# Quyida 08 uchun PATCH yo‘qsiz (Cell6–Cell8 patch qismini olib tashlagan) to‘liq tartiblangan kod.

In [1]:
# faers_...v2.csv qaysi root ostida bo‘lsa o‘shani PROJECT_ROOT qilib oladi:
# CELL 1 — Import + Path + Load (CSV bor joyni PROJECT_ROOT qiladi)
from pathlib import Path
import pandas as pd
import numpy as np

CWD = Path.cwd()

FNAME = "faers_25Q4_targets_multilabel_v2.csv"

# Qaysi papkalar ichidan qidiramiz
RAW_FOLDER_CANDIDATES = ["Raw_data", "RaW_data", "raw_data", "RAW_DATA"]
DATA_FOLDER_CANDIDATES = ["Data", "data", "DATA"]

PROJECT_ROOT = None
IN_CSV = None

checked = []

# 1) CWD dan yuqoriga qarab CSV ni aniq joyidan topamiz
for p in [CWD] + list(CWD.parents):
    for data_name in DATA_FOLDER_CANDIDATES:
        for raw_name in RAW_FOLDER_CANDIDATES:
            cand = p / data_name / raw_name / FNAME
            checked.append(cand)
            if cand.exists():
                PROJECT_ROOT = p
                IN_CSV = cand
                break
        if IN_CSV is not None:
            break
    if IN_CSV is not None:
        break

# 2) Topilmasa: project ichida kengroq qidiruv (faqat parents ichida)
if IN_CSV is None:
    # parents ichida o‘sha fayl nomini qidiramiz (kengroq)
    # Notebooks/Data bo‘sh bo‘lsa ham, yuqoridagi topishi kerak edi.
    # Baribir topilmasa — demak fayl bu tree ichida yo‘q.
    raise FileNotFoundError(
        "CSV topilmadi (parents bo‘ylab qidirildi).\n\n"
        f"CWD: {CWD.resolve()}\n\n"
        "Tekshirilgan namunaviy yo‘llar (oxirgi 10 ta):\n"
        + "\n".join([str(x) for x in checked[-10:]]) +
        "\n\nYECHIM:\n"
        "1) CSV haqiqatan ham shu project ichidami? (Downloads/Desktop/OneDrive boshqa joy bo‘lishi mumkin)\n"
        "2) Faylni project root ostiga ko‘chiring: <PROJECT>/Data/Raw_data/ ichiga\n"
        "3) Yoki fayl nomi boshqa bo‘lsa (patched/improved), FNAME ni moslang."
    )

print("PROJECT_ROOT:", PROJECT_ROOT.resolve())
print("IN_CSV:", IN_CSV.resolve())

# 3) Output dir (08 uchun)
OUT_DIR = PROJECT_ROOT / "Data" / "Processed" / "improvement_preprocess_v2"
OUT_DIR.mkdir(parents=True, exist_ok=True)
print("OUT_DIR:", OUT_DIR.resolve())

# 4) Load CSV (encoding fallback)
try:
    df = pd.read_csv(IN_CSV, low_memory=False, encoding="utf-8")
except UnicodeDecodeError:
    df = pd.read_csv(IN_CSV, low_memory=False, encoding="latin1")

print("Loaded:", df.shape)
df.head(2)

PROJECT_ROOT: C:\Users\xolmu\OneDrive\Desktop\Modul Program oyi\Modul_Program3\6_project_dori_tasiri_extract
IN_CSV: C:\Users\xolmu\OneDrive\Desktop\Modul Program oyi\Modul_Program3\6_project_dori_tasiri_extract\Data\Raw_data\faers_25Q4_targets_multilabel_v2.csv
OUT_DIR: C:\Users\xolmu\OneDrive\Desktop\Modul Program oyi\Modul_Program3\6_project_dori_tasiri_extract\Data\Processed\improvement_preprocess_v2
Loaded: (308959, 75)


,primaryid,caseid,caseversion,i_f_code,event_dt,mfr_dt,init_fda_dt,fda_dt,rept_cod,auth_num,...,y_infections,y_psychiatric,y_urinary,y_injection_site,y_ocular_visual,y_metabolic_endocrine,y_hypersensitivity_allergy,y_injury_accident,y_pregnancy_reproductive,y_labels
0,100324053,10032405,3,F,NaN,20251110,20140320,20251120,EXP,NaN,...,0,0,0,0,0,0,0,0,0,NaN
1,1012809821,10128098,21,F,20140219.0,20251003,20140428,20251010,EXP,NaN,...,1,0,0,1,0,1,1,0,0,cardiovascular; respiratory; general_systemic;...


In [2]:
# =========================================================
# CELL 2 — Columns + Basic Cleaning
# =========================================================
import pandas as pd
import numpy as np

# 1) Target ustunlari: y_ bilan boshlanadi, lekin y_labels emas
y_cols = sorted([c for c in df.columns if c.startswith("y_") and c != "y_labels"])

# 2) Text column (v2)
TEXT_COL = "REAC_pt_symptom_v2"
CLEAN_COL = "REAC_pt_symptom_v2_clean"

if TEXT_COL not in df.columns:
    raise ValueError(f"Input text ustuni topilmadi: {TEXT_COL}")

print("Num labels:", len(y_cols))
print("Labels:", y_cols)

# --- Text tozalash (faqat strip, keyin CELL 3 da to‘liq clean qilamiz) ---
df[TEXT_COL] = df[TEXT_COL].fillna("").astype(str).str.strip()
print("Empty text rows:", int((df[TEXT_COL] == "").sum()))

# --- Y ni 0/1 ga majburan tozalash (xavfsiz) ---
for c in y_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")  # '0'/'1' string bo‘lsa -> numeric

df[y_cols] = df[y_cols].fillna(0).astype(int)

# Y matritsa
Y = df[y_cols].copy()

# y_sum (0-label audit)
df["y_sum"] = Y.sum(axis=1).astype(int)
print("0-label rows:", int((df["y_sum"] == 0).sum()))

Num labels: 21
Labels: ['y_cardiovascular', 'y_dermatologic', 'y_edema_swelling', 'y_gastrointestinal', 'y_general_systemic', 'y_hematologic', 'y_hepatic', 'y_hypersensitivity_allergy', 'y_infections', 'y_injection_site', 'y_injury_accident', 'y_metabolic_endocrine', 'y_musculoskeletal', 'y_neurological', 'y_ocular_visual', 'y_pain_general', 'y_pregnancy_reproductive', 'y_psychiatric', 'y_renal', 'y_respiratory', 'y_urinary']
Empty text rows: 0
0-label rows: 59209


In [3]:
# =========================================================
# CELL 3 — Text CLEAN (faqat ';' bo‘yicha, comma bo‘yicha split YO‘Q)
# =========================================================
import re

_split_semicolon = re.compile(r"\s*;\s*")
_multi_space = re.compile(r"\s+")

def normalize_symptom_text(s: str) -> str:
    if s is None:
        return ""
    s = str(s).strip()
    if not s:
        return ""
    parts = _split_semicolon.split(s)  # faqat ';'
    out = []
    seen = set()
    for t in parts:
        t = _multi_space.sub(" ", t.strip())
        if not t:
            continue
        k = t.lower()
        if k in seen:
            continue
        seen.add(k)
        out.append(t)
    return "; ".join(out)

df[CLEAN_COL] = df[TEXT_COL].apply(normalize_symptom_text)

print("CLEAN_COL empty rows:", int((df[CLEAN_COL] == "").sum()))
df[[TEXT_COL, CLEAN_COL]].head(5)

CLEAN_COL empty rows: 0


,REAC_pt_symptom_v2,REAC_pt_symptom_v2_clean
0,Meningitis pneumococcal,Meningitis pneumococcal
1,Injection site reaction; General physical heal...,Injection site reaction; General physical heal...
2,Internal haemorrhage; Injury; Pain; Depression...,Internal haemorrhage; Injury; Pain; Depression...
3,Cytomegalovirus infection,Cytomegalovirus infection
4,Asthenia; Infusion related reaction; Blood pre...,Asthenia; Infusion related reaction; Blood pre...


In [4]:
# =========================================================
# CELL 4 — y_labels ni y_* dan qayta yasash (label_rules'ga tegmaydi)
# =========================================================
label_names = [c.replace("y_", "", 1) for c in y_cols]

def rebuild_y_labels(row_vals: np.ndarray) -> str:
    # row_vals: 0/1 array
    idxs = np.where(row_vals == 1)[0]
    if len(idxs) == 0:
        return np.nan
    labs = [label_names[i] for i in idxs]
    return "; ".join(labs)

arr = df[y_cols].to_numpy(dtype=np.int8)
df["y_labels"] = [rebuild_y_labels(arr[i]) for i in range(arr.shape[0])]

# sanity
bad1 = int(((df["y_sum"] == 0) & (df["y_labels"].notna())).sum())
bad2 = int(((df["y_sum"] > 0) & (df["y_labels"].isna())).sum())
print("Sanity bad (0 but has labels):", bad1)
print("Sanity bad (>0 but NaN labels):", bad2)

df[["y_sum", "y_labels"]].head(5)

Sanity bad (0 but has labels): 0
Sanity bad (>0 but NaN labels): 0


,y_sum,y_labels
0,0,NaN
1,8,cardiovascular; general_systemic; hypersensiti...
2,5,hematologic; infections; injury_accident; pain...
3,1,infections
4,13,cardiovascular; gastrointestinal; general_syst...


In [5]:
# =========================================================
# CELL 5 — 0-label top terms (audit uchun)
# =========================================================
def split_terms(clean_text: str):
    if clean_text is None:
        return []
    s = str(clean_text).strip()
    if not s:
        return []
    return [t.strip() for t in _split_semicolon.split(s) if t.strip()]

zero_terms = (
    df.loc[df["y_sum"] == 0, CLEAN_COL]
      .apply(split_terms)
      .explode()
      .dropna()
)

zero_terms = zero_terms[zero_terms.astype(str).str.strip() != ""]
vc = zero_terms.astype(str).str.lower().str.strip().value_counts()

top = vc.head(2000).reset_index()
top.columns = ["term_lc", "freq"]

ZERO_TERMS_PATH = OUT_DIR / "zero_label_top_terms_v1.csv"
top.to_csv(ZERO_TERMS_PATH, index=False)

print("Saved:", ZERO_TERMS_PATH)
top.head(20)

Saved: c:\Users\xolmu\OneDrive\Desktop\Modul Program oyi\Modul_Program3\6_project_dori_tasiri_extract\Data\Processed\improvement_preprocess_v2\zero_label_top_terms_v1.csv


,term_lc,freq
0,hypoacusis,285
1,knee arthroplasty,258
2,multiple sclerosis,256
3,respiratory failure,251
4,drug use disorder,251
5,coma,250
6,skin cancer,248
7,epilepsy,248
8,decreased appetite,235
9,multiple sclerosis relapse,231


In [6]:
# =========================================================
# CELL 6 — Save improved dataset (faqat text clean + y_labels rebuild)
# =========================================================
OUT_CSV = OUT_DIR / "faers_25Q4_targets_multilabel_v2_textclean_v1.csv"

# xohlasangiz y_sum ni ham qoldiramiz (EDA uchun foydali)
df.to_csv(OUT_CSV, index=False)

print("Saved dataset:", OUT_CSV)
print("Final shape:", df.shape)

Saved dataset: c:\Users\xolmu\OneDrive\Desktop\Modul Program oyi\Modul_Program3\6_project_dori_tasiri_extract\Data\Processed\improvement_preprocess_v2\faers_25Q4_targets_multilabel_v2_textclean_v1.csv
Final shape: (308959, 77)
